# 최적의 수소충전소 입지 선정


LPG 충전소와 주유소의 위치 정보들을 기반으로 최적의 수소충전소 입지를 선정하고자 한다. 이는 다음 순서로 진행된다.

1. LPG 충전소와 주유소의 위치(위도, 경도)에 따른 독립변수를 측정한다.

2. 저장된 랜덤 포레스트 모델을 통해 LPG 충전소와 주유소의 적합도를 평가한다.

3. 가장 적합한 5개의 위치를 분석한다.


In [1]:
# Load Common Libraries
import pandas as pd
from tqdm import tqdm
from typing import List, Tuple, Dict, Optional, Any, Union
import warnings
import numpy as np
import random
import os

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
# Define Path
main_path = os.getcwd()
original_dataset_path = os.path.join(main_path, 'data')
dataset_path = os.path.join(main_path, 'trans_data')

# grid_centers_path = os.path.join(dataset_path, 'grid_centers.csv')
lpg_station_path = os.path.join(dataset_path, 'lpg_station.csv')
gas_station_path = os.path.join(dataset_path, 'gas_station.csv')

# 거리 기반 데이터
hydrogen_stations_path = os.path.join(dataset_path, 'hydrogen_station.csv')
baby_school_location_path = os.path.join(
    dataset_path, 'baby_school_location.csv')
school_location_path = os.path.join(dataset_path, 'school_location.csv')
elderly_center_path = os.path.join(dataset_path, 'elderly_center.csv')
fire_station_path = os.path.join(dataset_path, 'fire_station.csv')
rescue_station_path = os.path.join(dataset_path, 'rescue_station.csv')

# 교통량 및 인구 기반 데이터
traffic_data_path = os.path.join(dataset_path, 'traffic.csv')
hydrogen_car_count_path = os.path.join(dataset_path, 'hydrogen_car_count.csv')
population_density_path = os.path.join(dataset_path, 'population_density.csv')

# 결과를 저장할 파일 경로
combined_target_path = os.path.join(
    dataset_path, 'combined_targets.csv')

"""     =====================     Reference Data     =====================     """

seoul_detail_map_path = 'shp/seoul_submunicipalities.shp'
seoul_map_path = 'shp/seoul_municipalities.shp'
model_path = "random_forest_regressor_model.pkl"

## 독립 변수 측정

LPG 충전소와 주요소의 위치 정보를 기반으로 독립 변수를 측정한다. 이는 create_regression_data.ipynb에서 독립 변수를 측정한 것과 동일한 방법으로 진행된다.


In [3]:
df1 = pd.read_csv(lpg_station_path).loc[:, ['업소명', '소재지', 'lat', 'long']]
df2 = pd.read_csv(gas_station_path).loc[:, ['주유소명', '주소', 'lat', 'long']]
df1.columns = ['name', 'address', 'lat', 'long']
df2.columns = ['name', 'address', 'lat', 'long']
target_df = pd.concat([df1, df2]).reset_index(drop=True)
target_df_length = len(target_df)

print(f"Total Grid Centers: {target_df_length}")
target_df.iloc[random.sample(
    [i for i in range(target_df_length)], 10), :].sort_index().T

Total Grid Centers: 521


,131,132,142,243,301,317,341,344,349,464
name,동일주유소,현대오일뱅크(주)직영 아리랑주유소,㈜오만불 신동방주유소,박물관주유소,SK에너지㈜ 기린주유소,대성산업㈜노량진주유소,지에스칼텍스(주) 동서울주유소,재건에너지 재정제2주유소 고속셀프지점,구천면주유소,하계삼호주유소
address,서울특별시 성북구 보문로 85,서울특별시 성북구 아리랑로 96,서울특별시 성북구 월계로 84,서울특별시 강서구 양천로 53길 97(가양1동 26-1),서울특별시 영등포구 선유로 270 (양평동4가),서울특별시 동작구 노량진로 172(노량진동),서울특별시 강동구 천호대로 1456,서울특별시 강동구 천호대로 1246,서울특별시 강동구 구천면로 357,서울특별시 노원구 공릉로 294(하계동)
lat,37.582958,37.600536,37.613404,37.567499,37.536332,37.513554,37.545234,37.536417,37.550458,37.633589
long,127.020374,127.014212,127.038057,126.850302,126.898774,126.945015,127.170219,127.149372,127.138227,127.072325


## 종속 변수 설정 - 수소충전소까지의 거리


서울 내부의 각각의 격자점(grid_centers)에서 **가장 가까운 수소충전소까지의 거리**를 계산한다.

현재 설치된 수소충전소의 위치가 사회적 합의를 거친 최적의 위치라고 가정하였기에 수소충전소까지의 거리가 가까울수록 적합한 수소충전소 설치 위치로 판단한다.


In [4]:
import geopy.distance
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd


def find_nearest_station(location1: pd.DataFrame, location2: pd.DataFrame) -> List[float]:
    """
    가장 가까운 지점까지의 거리를 계산.

    Args:
    - location1: 가장 가까운 지점을 찾을 기준, lat과 long을 가지고 있는  pandas DataFrame
    - location2: 가장 가까운 지점을 찾을 대상체, lat과 long을 가지고 있는 pandas DataFrame

    Returns:
    - min_distnace_list: location1에서 가장 가까운 location2까지의 거리 리스트(거리 단위: km)
    """
    min_distnace_list = []
    for _, location1_row in tqdm(location1.iterrows(), total=len(location1)):

        min_distance = float('inf')
        for _, location2_row in location2.iterrows():

            distance = geopy.distance.geodesic(  # 거리 계산
                (location1_row['lat'], location1_row['long']), (location2_row['lat'], location2_row['long'])).km

            if distance < min_distance:  # 최소 거리 갱신
                min_distance = distance
        min_distnace_list.append(min_distance)
    return min_distnace_list


def get_nearest_point_info(location1: pd.DataFrame, location2: pd.DataFrame, col_name: str) -> List[Any]:
    """
    가장 가까운 수소 충전소까지의 거리를 계산.

    Args:
    - location1: 가장 가까운 지점을 찾을 기준, lat과 long을 가지고 있는  pandas DataFrame
    - location2: 가장 가까운 지점을 찾을 대상체, lat과 long을 가지고 있는 pandas DataFrame
    - col_name: 대상체에서 추출하고자 하는 정보의 컬럼명

    Returns:
    - data_list: 대상체에서 추출된 정보 리스트
    """
    if col_name not in location2.columns:
        raise ValueError(f"{col_name} is not in location2 columns")

    data_list = []
    for _, location1_row in tqdm(location1.iterrows(), total=len(location1)):

        min_distance = float('inf')
        min_distance_data = None
        for _, location2_row in location2.iterrows():
            distance = geopy.distance.geodesic(  # 거리 계산
                (location1_row['lat'], location1_row['long']), (location2_row['lat'], location2_row['long'])).km

            if distance < min_distance:  # 최소 거리 갱신
                min_distance = distance
                min_distance_data = location2_row[col_name]
        data_list.append(min_distance_data)
    return data_list


def find_region(gdf, latlong):
    point = Point(latlong[1], latlong[0])
    for idx, row in gdf.iterrows():
        if row['geometry'].contains(point):
            return row
    return None


def get_full_region(df: pd.DataFrame):
    gdf = gpd.read_file(seoul_detail_map_path)
    gdf_higher = gpd.read_file(seoul_map_path)

    result_list = []
    for idx, row in df.iterrows():
        result = find_region(gdf, (row['lat'], row['long']))
        result_higher = find_region(gdf_higher, (row['lat'], row['long']))
        if result is not None and result_higher is not None:
            result_list.append(f"{result_higher['name']} {result['name']}")
        else:
            result_list.append('')
    return result_list


# 열 이름과 파일 경로쌍
column_path_dict = {
    'school_distance': school_location_path,
    'baby_school_distance': baby_school_location_path,
    'elderly_center_distance': elderly_center_path,
    'fire_station_distance': fire_station_path,
    'rescue_station_distance': rescue_station_path,
}

for column, path in column_path_dict.items():
    print(f"Calculating {column}...")
    location = pd.read_csv(path)
    result_validated = find_nearest_station(target_df, location)
    target_df[column] = result_validated


traffic_data = pd.read_csv(traffic_data_path)
result_validated = get_nearest_point_info(
    target_df, traffic_data, 'traffic_volume')
target_df['traffic_volume'] = result_validated


hydrogen_car_count = pd.read_csv(hydrogen_car_count_path)
hydrogen_car_count.set_index('district', inplace=True)
hydrogen_car_count_dict = hydrogen_car_count.to_dict()['count']
hydrogen_car_count_dict['중랑구'] = hydrogen_car_count_dict['중량구']  # 오타 보정
full_region_list = get_full_region(target_df)
val_list = []
for region in full_region_list:
    head_region = region.split(' ')[0]
    if head_region in hydrogen_car_count_dict:
        val_list.append(hydrogen_car_count_dict[head_region])
    else:
        val_list.append(0)
target_df['hydrogen_car_count'] = val_list


population_density = pd.read_csv(population_density_path)
population_density.set_index('주소', inplace=True)
population_density_dict = population_density.to_dict()['인구 (명)']
val_list = [population_density_dict[full_region_list[i]] if full_region_list[i]
            in population_density_dict else 0 for i in range(len(full_region_list))]
target_df['population_density'] = val_list


target_df.iloc[random.sample(
    [i for i in range(target_df_length)], 10), :].sort_index()

Calculating school_distance...


100%|██████████| 521/521 [01:18<00:00,  6.64it/s]


Calculating baby_school_distance...


100%|██████████| 521/521 [04:20<00:00,  2.00it/s]


Calculating elderly_center_distance...


...

Calculating fire_station_distance...


100%|██████████| 521/521 [00:01<00:00, 357.52it/s]


Calculating rescue_station_distance...


100%|██████████| 521/521 [00:08<00:00, 62.97it/s]


,name,address,lat,long,school_distance,baby_school_distance,elderly_center_distance,fire_station_distance,rescue_station_distance,traffic_volume,hydrogen_car_count,population_density
57,(주)남양가스,서울특별시 양천구 남부순환로 538 남양가스충전소,37.520701,126.837427,0.367685,0.154816,0.098887,95.041268,91.847731,103550,126,19199
87,지에스칼텍스㈜ 도루코주유소,서울특별시 성동구 아차산로 180(성수동2가 19-14),37.542027,127.063423,0.257231,0.470327,0.348579,92.614347,90.415468,49392,69,10520
156,북서울고속주유소,서울특별시 강북구 삼양로 410,37.640237,127.017250,0.272993,0.249443,0.206232,81.404467,79.047352,46744,41,29916
228,양천구주유소,서울특별시 양천구 국회대로 275(목동),37.530373,126.865512,0.660000,0.233278,0.157453,93.736338,90.654299,67316,265,20514
249,목화주유소,서울특별시 강서구 국회대로 251(화곡4동 785-2),37.530055,126.862834,0.691610,0.242044,0.224410,93.791373,90.698287,67316,265,20514
272,현대오일뱅크(주)직영 신구로주유소,서울특별시 구로구 가마산로 293(구로동),37.497553,126.892394,0.155212,0.190840,0.170668,97.205790,94.242644,57000,200,31740
420,SK북악주유소,서울특별시 종로구 평창문화로137(평창동),37.609853,126.974865,0.765432,0.066617,0.185941,84.608337,82.021224,0,79,17858
433,에쓰-오일(주)오토테크주유소,서울특별시 서초구 효령로 356 (서초동),37.486497,127.023064,0.155259,0.120555,0.106994,98.465652,96.052264,229603,236,20622
457,월계주유소,서울특별시 노원구 월계로 252(월계동),37.624441,127.050758,0.356834,0.218104,0.179944,83.400938,81.206438,147543,84,26066
458,현대오일뱅크㈜ 직영 하계주유소,서울특별시 노원구 노원로17길 29(하계동),37.642216,127.070389,0.170596,0.312600,0.151879,81.635725,79.561035,37487,84,0


In [5]:
# nan 값 제거
target_df.dropna(inplace=True)

# 0을 가지는 값 제거
for col_name in ['hydrogen_car_count', 'population_density']:
    target_df = target_df[target_df[col_name] != 0]

# school_distnace와 baby_school_distance에서 7m 이하의 값을 가지는 행을 제거
target_df = target_df[(target_df['school_distance'] > 0.007) & (
    target_df['baby_school_distance'] > 0.007)]
print(f"Total targets: {len(target_df)}")

# 데이터 저장
target_df.reset_index(drop=True, inplace=True)
target_df.to_csv(combined_target_path, index=False)
print(f"Save to trans_data/combined_targets.csv")
target_df.tail(3)

Total targets: 502
Save to trans_data/combined_targets.csv


,name,address,lat,long,school_distance,baby_school_distance,elderly_center_distance,fire_station_distance,rescue_station_distance,traffic_volume,hydrogen_car_count,population_density
499,(주)송만에너지 도봉제일주유소,서울특별시 도봉구 도봉로 783 (도봉동),37.674474,127.044067,0.429071,0.113170,0.176624,77.816630,75.630924,87685,47,20622
500,노원교주유소,서울특별시 도봉구 마들로 776 (도봉동),37.679015,127.049751,0.550059,0.201737,0.098120,77.367618,75.217926,87685,47,26359
501,오복주유소,서울특별시 도봉구 방학로 43 (방학동),37.662280,127.047441,0.173071,0.225337,0.187990,79.193255,77.014791,37487,47,28410


In [6]:
# 독립변수 측정 및 데이터 전처리 과정에서 생성한 모든 변수 및 함수를 삭제함으로서 메모리를 확보한다.
%reset_selective -f a 

## 랜덤 포레스트 모델을 통한 평가


랜덤 포레스트 모델을 불러와 LPG 충전소와 주유소의 적합도를 평가한다.


In [7]:
# Define Path
main_path = os.getcwd()
original_dataset_path = os.path.join(main_path, 'data')
dataset_path = os.path.join(main_path, 'trans_data')

# 저장된 target의 경로
combined_target_path = os.path.join(
    dataset_path, 'combined_targets.csv')

"""     =====================     Reference Data     =====================     """

seoul_detail_map_path = 'shp/seoul_submunicipalities.shp'
seoul_map_path = 'shp/seoul_municipalities.shp'
model_path = "random_forest_regressor_model.pkl"

In [8]:
data = pd.read_csv(combined_target_path)

feature_names = ['school_distance', 'baby_school_distance', 'elderly_center_distance',
                 'fire_station_distance', 'rescue_station_distance', 'traffic_volume',
                 'hydrogen_car_count', 'population_density']
X = data[feature_names]
print(f"X shape: {X.shape}")

X shape: (502, 8)


In [9]:
os.path.join(os.getcwd(), os.listdir()[-3])

'/Users/jaewone/Downloads/Selecting-the-optimal-hydrogen-charging-station-location_copy/assets'

In [11]:
import joblib

# 저장된 모델 불러오기
loaded_model = joblib.load('random_forest_regressor_model.pkl')

# 불러온 모델로 예측 수행 (예: X의 첫 5개 샘플 예측)
predictions = loaded_model.predict(X)
data['predictions'] = predictions
data = data.sort_values(by='predictions')

# 예측 결과 저장
data.to_csv(os.path.join(dataset_path, 'predictions.csv'), index=False)

# 예측 결과 출력
print(f"Predictions shaoe: {predictions.shape}")
print(predictions[:5])

Predictions shaoe: (502,)
[4.6262864  4.5824228  4.56695034 4.5712483  4.62981194]


In [12]:
# 적합도가 높은 5개의 주유소 출력
data.head(5)

,name,address,lat,long,school_distance,baby_school_distance,elderly_center_distance,fire_station_distance,rescue_station_distance,traffic_volume,hydrogen_car_count,population_density,predictions
248,㈜한진김포공항주유소,서울특별시 강서구 하늘길 259(발산1동 423-2),37.550942,126.814720,0.250230,0.648268,0.528156,91.949983,88.649392,26957,265,30927,4.537419
424,현대오일뱅크(주)직영양재현대주유소,서울특별시 서초구 마방로 6 (양재동),37.473550,127.039695,0.322609,0.027857,0.551773,99.998973,97.650466,231970,236,42683,4.566835
222,지에스칼텍스(주)건지주유소,서울특별시 양천구 안양천로 1171(목동),37.545316,126.882153,0.267238,0.022099,0.514473,91.971160,88.955404,81224,265,41628,4.566835
2,대치에너지주식회사,서울특별시 강남구 남부순환로 3179,37.499458,127.078164,0.271657,0.534897,0.562998,97.451363,95.286921,38760,241,28358,4.566950
26,복지공릉충전소,서울특별시 노원구 화랑로 826 (공릉동),37.638936,127.109652,0.495829,0.557219,2.361153,82.496516,80.626527,51450,84,42538,4.567165


In [13]:
# 적합도가 낮은 5개의 주유소 출력
data.tail(5)

,name,address,lat,long,school_distance,baby_school_distance,elderly_center_distance,fire_station_distance,rescue_station_distance,traffic_volume,hydrogen_car_count,population_density,predictions
408,구도일주유소 특종,서울특별시 종로구 평창문화로 90(평창동),37.606712,126.971047,0.298149,0.254872,0.394363,84.950425,82.343598,0,79,17858,7.223275
124,장위주유소,서울특별시 성북구 화랑로 110,37.603464,127.043444,0.577234,0.294098,0.255025,85.656548,83.407895,0,73,19739,7.321732
405,안풍주유소,서울특별시 종로구 자하문로 303(홍지동),37.599210,126.959168,0.256094,0.429915,0.431019,85.771482,83.105041,0,79,9536,7.323167
406,자하문주유소,서울특별시 종로구 자하문로 248(부암동),37.596301,126.964348,0.815781,0.590367,0.141831,86.097770,83.455719,0,79,9536,7.336416
30,(주)홍릉가스충전소,서울특별시 동대문구 회기로 44,37.591134,127.041481,0.496813,0.342291,0.360102,87.004101,84.736565,0,73,40111,7.401659
